Karina Chiñas Fuenes | 18/04/2023 | Universiteit Gent

In [ ]:
%load_ext autoreload
%autoreload 2

This notebook is based on the document:

* _Feature Selection for Clustering_, [Manoranjan Dash and Huan Liu](https://www.public.asu.edu/~huanliu/papers/pakdd00clu.pdf), 2000.

# NO RESEARCH VARIABLES

This notebook helps me understand if there is a structure among the variables of interest. 

In [ ]:
import sys
sys.path.append('../')
import tokamakTK
from tokamakTK import get_regression


import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer

pd.set_option('display.max_columns', None)
path = "../data/"

In [ ]:
# Obtained from Optimization

min_subset_ids = pd.read_csv(path+"R_ids_alpha_0.6357.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label"] = 1


print(
    f"{ round( (len(min_subset_ids)/len(DB5))*100     ,2)  }% of the data decreased alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids)/len(DB5))*100 ,2)  }% of the data did not decrease alpha_R"
)

In [ ]:
get_regression( DB5[DB5.label.isin([1])], DB2  )[1].summary()

## Treatment to Data and Setting Features of Interest

In [ ]:
plasma_characteristics = ["BEIMHD", "BETASTAR", "CONFIG", "DWDIA", "EPS", "HYBRID", "IP", "KAREA",
                          "LCOULOMB", "LHTIME", "MEFF", "NUSTAR", "PREMAG", "QCYL5", "RHOSTAR",
                          "TORQ", "VOL", "WMHD"
                         ]
TOK_characteristics = ["AMIN", "BT", "DIVMAT", "DIVNAME", "LIMMAT", "TOK", "WALMAT"]
ELM = ["ELMFREQ", "ELMTYPE"]
heating = ["AUXHEAT", "ECHMODE", "ICSCHEME", "PECRH", "PELLET", "PICRH"]
impurities = ["ENBI", "EVAP", "PNBI", "POHM", "PRAD", "ZEFF", "ZEFFNEO"]
power = ["PFLOSS", "PLTH"]
temperatures = ["TAV", "TEV", "TIV"]
fast_particles = ["NEL", "NESOL", "OMEGACYCL", "WFFORM", "WFICFORM"]

features = (plasma_characteristics + TOK_characteristics + ELM + heating + 
                       impurities + power + temperatures  + fast_particles)

data = DB5[DB5.label.isin([1])].copy()
data = data[features+["DATE"]].reset_index(drop=True)

# Needed to respectively clean each dtype
num_features = data.select_dtypes(include=['int', 'float']).columns.tolist()
cat_features = data.select_dtypes(include=['object']).columns.tolist()
cat_features = cat_features[:-1] # Removing "DATE"

data_num = data[num_features+["TOK"]]
data_cat = data[cat_features]

In [ ]:
DB5 = tokamakTK.clean_categorical_data(DB5)

# Needed to respectively clean each dtype
num_features = DB5[features].select_dtypes(include=['int', 'float']).columns.tolist()
cat_features = DB5[features].select_dtypes(include=['object']).columns.tolist()

data_num = DB5[num_features+["TOK","DATE"]]
data_cat = DB5[cat_features]

data_num = tokamakTK.clean_numerical_data(data_num)
data = pd.concat([data_cat, data_num], axis=1)

data = data[sorted(data.columns)]

In [ ]:
N,M = data.shape

# used to get feature ranking
feature_dic = dict( zip( list(range(M)), data.columns ) )

## ENTROPY IN DATASET | Categorical
#### Using the concept of entropy to rank the importance of features.

0. Computing the distance between two points

Euclidean distance:

$$
    D^k_{ij} = \left( \frac{ x_{ik} - x_{jk} }{ max(F_k) - min(F_k) } \right)^2 
$$

$$
    D_{ij} = \left[ \sum^M_{k=1} \: D^k_{ij}\right]^{1/2}
$$

Where $F_k$ is the $k$-th column representing the $k$-th feature. With $k=1,\dots,M$ The $x_{ik}$ is the $i$-th point of the $k$-th feature; the same for $j$. With $i$,$j=1\ldots,N$. And $N$ being the total points of observations. Thus the similarity between two points

Defined as:

$$
    S_{ij} = e^{-\alpha \cdot D_{ij}}
$$

Where $\alpha$ is a parameter. Here $\alpha = 0.5$. According to [Manoranjan Dash and Huan Liu](https://www.public.asu.edu/~huanliu/papers/pakdd00clu.pdf), If one plots similarity vs distance, the curve will have a biggercurvature for a larger $\alpha$. 

1. Computing the similarity between two points with Humming distance:

\begin{equation}
        S_{ij} = \frac{1}{M}\sum^M_{k=1} \delta_{ij}(x^k); \quad \text{with} \:\: \delta_{ij}(x^k) = \begin{cases}
    1,\:\text{if}\:\: x^k_i = x^k_j\\
    0,\:\text{if}\:\: x^k_i \neq x^k_j
    \end{cases}
\end{equation}

With $M$ being the total number of features.


2. The entropy of the dataset is:

$$
    E = -\sum^N_{i=1}\sum^N_{j=1} E_{ij}= -\sum^N_{i=1}\sum^N_{j=1}\left[ S_{ij}\cdot\text{Log}\left(S_{ij}\right) +  \left(1-S_{ij}\right)\cdot\text{Log}\left(1-S_{ij}\right) \right]
$$

In the expression above, $N$ corresponds to the total number of observations in dataset (rows). 

The ranking is done in an interative process: eliminate a column at a time and assess the disorder that the removal has caused by calculating the corresponding entropy. Let us sat that $E_{^-F_i} > E_{^-F_j}$ is the statement that the removal of feature $i$ of the dataset caused more disorder than the removal of feature $j$. One can then construct an array $\vec{E_-} = \{E_{F_k}\}$, for $k=1,\ldots,M$. Thus, the most important feature will be  max$(\vec{E_-})$.

$\delta_{ij}(x^k)\:\rightarrow$ `np.frompyfunc( lambda x,y: x-y, 2, 1).reduce( np.array( np.meshgrid(F_k,F_k) ) )`

In [ ]:
features_ranked_entropy = tokamakTK.get_ranked_features(data)
ordered_features = list(features_ranked_entropy.index.map(feature_dic))

In [ ]:
plt.figure(figsize=(6,3))
plt.scatter(features_ranked_entropy.index, features_ranked_entropy.values, c="r")
plt.xlabel("Removed Feature")
plt.ylabel("Entropy of Dataset");

In [ ]:
print(ordered_features,)

### Comparing with the case not containing research variables

```Python
['ZEFFNEO', 'EPS', 'WFICFORM', 'BETASTAR', 'RHOSTAR', 'DWDIA', 'KAREA', 'NEL', 'OMEGACYCL', 'ENBI', 'WMHD', 'AMIN', 'TAV', 'WFFORM', 'PICRH', 'PLTH', 'MEFF', 'QCYL5', 'NUSTAR', 'PNBI', 'VOL', 'LCOULOMB', 'LHTIME', 'PFLOSS', 'BT', 'BEIMHD', 'POHM', 'ZEFF', 'PECRH', 'ELMFREQ', 'PRAD', 'NESOL', 'TORQ', 'TOK', 'WALMAT', 'LIMMAT', 'HYBRID', 'EVAP', 'ECHMODE', 'PREMAG', 'DIVMAT', 'PELLET', 'ELMTYPE', 'ICSCHEME', 'AUXHEAT', 'CONFIG']
```